# How this works?

Here on Wikipedia, [it is said](https://en.wikipedia.org/wiki/Point_in_polygon) that the most known solution to this kind of problem is called the **even–odd rule algorithm**:

> One simple way of finding whether the point is inside or outside a simple polygon is to test how many times a ray, starting from the point and going in any fixed direction, intersects the edges of the polygon. If the point is on the outside of the polygon the ray will intersect its edge an even number of times. If the point is on the inside of the polygon then it will intersect the edge an odd number of times. The status of a point on the edge of the polygon depends on the details of the ray intersection algorithm.

However, the examples show that a point is considered inside even if it is actually on the edge. Therefore, we must find these points ourselves first. hjh2005 writes [a post](https://blog.csdn.net/libaineu2004/article/details/102940607?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161502297916780266287379%2522%252C%2522scm%2522%253A%252220140713.130102334.pc%255Fall.%2522%257D&request_id=161502297916780266287379&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~first_rank_v2~rank_v29-2-102940607.first_rank_v2_pc_rank_v29&utm_term=%E5%A6%82%E4%BD%95%E5%88%A4%E6%96%AD%E4%B8%80%E4%B8%AA%E7%82%B9%E6%98%AF%E5%90%A6%E5%9C%A8%E5%A4%9A%E8%BE%B9%E5%BD%A2%E5%86%85%EF%BC%9F+python) (in Chinese) explaining how this idea works with Python. And there is also Python package called Shapely to specifically solve this problem but it is not working well with my computer due to some architecture issue, sadly. 

In [2]:
install.packages("sp")
library(sp)

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



We are to use point.in.polygon in [this package](https://edzer.github.io/sp/):

> Pebesma EJ, Bivand RS (2005). “Classes and methods for spatial data in R.” R News, 5(2), 9–13. https://CRAN.R-project.org/doc/Rnews/. <br>Bivand RS, Pebesma E, Gomez-Rubio V (2013). Applied spatial data analysis with R, Second edition. Springer, NY. https://asdar-book.org/.




The function's source code is available [here](https://rdrr.io/cran/sp/src/R/point.in.polygon.R)(a link to rdrr.io). According to this page, the function calls InPoly function, which can be found on [Github](https://github.com/cran/ripa/blob/master/src/inpoly.c) and described in "Computational Geometry in C" (Second Edition), Chapter 7, as shown below.

<img src="https://txcdn.ydchen.cn/assignments/Screenshot%202021-03-08%20at%2020.41.06.png" alt="PIP" style="zoom:50%;" />

<img src="https://txcdn.ydchen.cn/assignments/Screenshot%202021-03-09%20at%2006.43.30.png" alt="PIP" style="zoom:50%;" />

Maybe I will try later how to implement this with Python, but I am running out of time...

# Have a try first

Doc: https://rdrr.io/cran/sp/man/point.in.polygon.html

In [3]:
# point.in.polygon(point.x, point.y, pol.x, pol.y, mode.checked=FALSE)
# open polygon:
# point.in.polygon(1:10,1:10,c(3,5,5,3),c(3,3,5,5))
# closed polygon:
# point.in.polygon(1:10,rep(4,10),c(3,5,5,3,3),c(3,3,5,5,3))

**What does this means?**
> integer array; values are: 0: point is strictly exterior to pol; 1: point is strictly interior to pol; 2: point lies on the relative interior of an edge of pol; 3: point is a vertex of pol.

**Examples**

In [4]:
point.in.polygon(3,3,c(1,1,10,10),c(1,5,5,1))

[1] 1

In [5]:
point.in.polygon(7,5,c(1,1,10,10),c(1,5,5,1))

[1] 2

Should be on the edge here... A bit tricky.

In [6]:
point.in.polygon(5,6,c(1,1,10,10),c(1,5,5,1))

[1] 0

In [7]:
point.in.polygon(c(3,7,5),c(3,5,6),c(1,1,10,10),c(1,5,5,1))

[1] 1 2 0

# Officially start to solve the question

In [8]:
getwd() # Let's see where we are

[1] "/Users/ydchen/Desktop/University applications/NTU-interview-questions/original/Question 6"

In [9]:
polygon_coords <- read.table("input_question_6_polygon")
#polygon_coords
poly_x <- as.numeric(unlist(polygon_coords[1]))
poly_y <- as.numeric(unlist(polygon_coords[2]))
poly_x
poly_y

[1]  4  2  3  2  5  9 14 20 18 11

[1]  3  6 12 17 20 21 19 14  3  7

In [10]:
points_coords <- read.table("input_question_6_points")
#points_coords
point_x <- as.numeric(unlist(points_coords[1]))
point_y <- as.numeric(unlist(points_coords[2]))
point_x
point_y

[1]  7 10 11 12 16 16 17 18 18 20

[1] 11 14  4 21  3 10  4  7 17  7

In [11]:
results <- point.in.polygon(point_x,point_y,poly_x,poly_y)
results 

[1] 1 1 0 0 0 1 1 1 0 0

In [12]:
for (i in 1:length(results)){
    if (results[i] > 0){
        results[i] <- "inside"
    }else{
        results[i] <- "outside"
    }
}
results

[1] "inside"  "inside"  "outside" "outside" "outside" "inside"  "inside" 
 [8] "inside"  "outside" "outside"

In [13]:
# output example
#3 3 inside
#7 5 inside
#5 6 outside

In [14]:
results_list<-t(rbind(as.list(point_x), as.list(point_y), as.list(results)))
results_list

7,11,inside
10,14,inside
11,4,outside
12,21,outside
16,3,outside
16,10,inside
17,4,inside
18,7,inside
18,17,outside
20,7,outside


In [215]:
write.table(results_list, 
            file = "output_question_6.txt", 
            sep = " ", 
            row.names = FALSE, 
            col.names = FALSE)

In [16]:
#help(write.table)